In [ ]:
# Pancreatic Cancer Detection Notebook

In [5]:
import pandas as pd
import numpy as np

# Data Loading and Preprocessing 

Ideas we should probably do for both datasets:
    - load data. remove any Null/NaN data, see if there are negative values and remove those if applicable.
    - if blank data cells, impute the data. scale the appropriate data columns


    - split the dataset into X and y subsets for passing into models
        - which really means also split into 80/20; 80% training(validation included) and 20% test

    - Note: we should do some data visualizations --> heatmaps? histograms of label distributions for proof
        - of good splits. mutual information??? (need to better understand if its useful)


In [6]:
# Loading urinary biomarker dataset
urinary_df = pd.read_csv('urinaryBiomarkerData.csv')
urinary_df = urinary_df.drop(columns=['sample_id','patient_cohort','sample_origin','age','sex','stage','benign_sample_diagnosis','plasma_CA19_9','REG1A'])
print(urinary_df)

     diagnosis  creatinine     LYVE1       REG1B         TFF1
0            1     1.83222  0.893219   52.948840   654.282174
1            1     0.97266  2.037585   94.467030   209.488250
2            1     0.78039  0.145589  102.366000   461.141000
3            1     0.70122  0.002805   60.579000   142.950000
4            1     0.21489  0.000860   65.540000    41.088000
..         ...         ...       ...         ...          ...
585          3     0.52026  7.058209  156.241000   525.178000
586          3     0.85956  8.341207   16.915000   245.947000
587          3     1.36851  7.674707  289.701000   537.286000
588          3     1.33458  8.206777  205.930000   722.523000
589          3     1.50423  8.200958  411.938275  2021.321078

[590 rows x 5 columns]


In [15]:
# Preprocessing urinary biomarker dataset
# Note: There are no negative or Null/NaN values in the dataset
    # So, no need to remove samples or impute the data
    # Only need to scale the appropriate data columns
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
val_cols = urinary_df.loc[:, ~urinary_df.columns.isin(['diagnosis'])]
urinary_df[val_cols.columns] = scaler.fit_transform(val_cols)
print(urinary_df)

     diagnosis  creatinine     LYVE1     REG1B      TFF1
0            1    1.529927 -0.631661 -0.299975  0.055876
1            1    0.183680 -0.298597 -0.088256 -0.384680
2            1   -0.117454 -0.849256 -0.047976 -0.135425
3            1   -0.241451 -0.890812 -0.261065 -0.450584
4            1   -1.003143 -0.891378 -0.235767 -0.551475
..         ...         ...       ...       ...       ...
585          3   -0.524871  1.162636  0.226755 -0.071998
586          3    0.006542  1.536048 -0.483726 -0.348568
587          3    0.803662  1.342066  0.907324 -0.060005
588          3    0.750521  1.496923  0.480141  0.123466
589          3    1.016227  1.495229  1.530663  1.409888

[590 rows x 5 columns]


In [26]:
# Split data into training/validation and test (80/20)
urinary_df = urinary_df.sample(frac=1) # randomly shuffle data
eighty_percent = int(len(urinary_df) * 0.8)
train_urinary_data = urinary_df.loc[:eighty_percent-1]
test_urinary_data = urinary_df.loc[eighty_percent:]
print("train_urinary_data: \n", train_urinary_data)
print("test_urinary_data: \n", test_urinary_data)

train_urinary_data: 
      diagnosis  creatinine     LYVE1     REG1B      TFF1
242          2    2.025912 -0.859894 -0.527933 -0.424243
42           1    0.307676  1.233065  0.033406  0.819211
448          3   -1.215708 -0.408099 -0.505832 -0.211488
117          1   -0.223737 -0.890799 -0.569977 -0.553488
331          2    0.077397 -0.747354 -0.302675  0.266234
..         ...         ...       ...       ...       ...
543          3   -0.967715  2.200545  3.062721  0.185451
495          3   -0.542585 -0.243795 -0.446711 -0.239743
98           1    0.165966 -0.810302  0.136955 -0.422724
66           1   -1.073998 -0.891431 -0.565114 -0.592159
471          3   -1.180281  0.339548 -0.285027 -0.547370

[154 rows x 5 columns]
test_urinary_data: 
      diagnosis  creatinine     LYVE1     REG1B       TFF1
472          3   -0.737436  0.740629  0.132556   1.122018
268          2   -0.347733 -0.728078 -0.398214  -0.242570
19           1   -0.542585 -0.721946 -0.500534  -0.327524
180          1   

# Model Creation
5 classifiers: Random Forest Classifier. XGBoost, Support Vector Machine, Gaussian Naive Bayes, and K Neighbors Classifier
- we want to initialize all 5 models, probably default hyperparameters for most values.
    - it's 5 models per dataset, so we'll have 10 in total

In [24]:
# ! pip3 install xgboost

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 2.2 MB 3.6 MB/s eta 0:00:01
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.


In [27]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
import xgboost as xgb
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier

In [28]:
# Max: add/remove/adjust hyperparameters as you see fit
rf = RandomForestClassifier(n_estimators=100, random_state=42)
xgb = xgb.XGBClassifier(n_estimators=100, random_state=42, learning_rate=0.01)
svm = SVC(C=1, kernel="rbf", random_state=42)
gnb = GaussianNB()
knn = KNeighborsClassifier()

# Training and Validation

- Train the models using 10-fold cross validation 
    - we can probably do this in 2 separate cross-validation loops. 
        - first one is for urinary dataset and we train/validate all 5 models (get their scores and loss? too)
        - repeat for second dataset

In [30]:
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

rf_scores = []
xgb_scores = []
svm_scores = []
gnb_scores = []
knn_scores = []

X = train_urinary_data
kf = KFold(n_splits=10)

for train_index, val_index in kf.split(X): #10 folds
    # set up training data
    train_chunk = X.iloc[train_index]
    y_train = train_chunk['diagnosis']
    X_train = train_chunk.drop(columns=['diagnosis'])

    # set up validation data
    val_chunk = X.iloc[val_index]
    y_val = val_chunk['diagnosis']
    X_val = val_chunk.drop(columns=['diagnosis'])

    # Random Forest
    rf.fit(X_train, y_train)
    rf_pred = rf.predict(X_val)
    rf_score = accuracy_score(y_val, rf_pred)
    rf_scores.append(rf_score)

    # XGBoost
    le = LabelEncoder() # use to convert classes from [1,2,3] to [0,1,2]
    xgb_y_train = le.fit_transform(y_train)
    xgb.fit(X_train, xgb_y_train)
    xgb_pred = xgb.predict(X_val)
    xgb_pred = le.inverse_transform(xgb_pred)
    xgb_score = accuracy_score(y_val, xgb_pred)
    xgb_scores.append(xgb_score)

    # SVM
    svm.fit(X_train, y_train)
    svm_pred = svm.predict(X_val)
    svm_score = accuracy_score(y_val, svm_pred)
    svm_scores.append(svm_score)

    # Gaussian Naive Bayes
    gnb.fit(X_train, y_train)
    gnb_pred = gnb.predict(X_val)
    gnb_score = accuracy_score(y_val, gnb_pred)
    gnb_scores.append(gnb_score)

    # KNearestNeighbors
    knn.fit(X_train, y_train)
    knn_pred = knn.predict(X_val)
    knn_score = accuracy_score(y_val, knn_pred)
    knn_scores.append(knn_score)

print("RF training accuracies: ", rf_scores)
print("RF mean: ", np.mean(rf_scores))
print("RF standard dev: ", np.std(rf_scores), "\n")

print("XGB training accuracies: ", xgb_scores)
print("XGB mean: ", np.mean(xgb_scores))
print("XGB standard dev: ", np.std(xgb_scores), "\n")

print("SVM training accuracies: ", svm_scores)
print("SVM mean: ", np.mean(svm_scores))
print("SVM standard dev: ", np.std(svm_scores), "\n")

print("GNB training accuracies: ", gnb_scores)
print("GNB mean: ", np.mean(gnb_scores))
print("GNB standard dev: ", np.std(gnb_scores), "\n")

print("KNN training accuracies: ", knn_scores)
print("KNN mean: ", np.mean(knn_scores))
print("KNN standard dev: ", np.std(knn_scores), "\n")

RF training accuracies:  [0.5625, 0.5, 0.625, 0.5625, 0.6, 0.5333333333333333, 0.6666666666666666, 0.5333333333333333, 0.5333333333333333, 0.6666666666666666]
RF mean:  0.5783333333333334
RF standard dev:  0.055646453415988485 

XGB training accuracies:  [0.5, 0.5625, 0.4375, 0.625, 0.6, 0.5333333333333333, 0.6666666666666666, 0.6, 0.6, 0.4]
XGB mean:  0.5525
XGB standard dev:  0.08047601437005245 

SVM training accuracies:  [0.6875, 0.5625, 0.625, 0.6875, 0.7333333333333333, 0.4, 0.6, 0.6, 0.5333333333333333, 0.6]
SVM mean:  0.6029166666666665
SVM standard dev:  0.08915206266698363 

GNB training accuracies:  [0.625, 0.5625, 0.625, 0.6875, 0.5333333333333333, 0.2, 0.7333333333333333, 0.5333333333333333, 0.5333333333333333, 0.4666666666666667]
GNB mean:  0.55
GNB standard dev:  0.13935615841751983 

KNN training accuracies:  [0.4375, 0.5, 0.5625, 0.5625, 0.6, 0.5333333333333333, 0.6666666666666666, 0.5333333333333333, 0.3333333333333333, 0.6666666666666666]
KNN mean:  0.539583333333333

# Testing Models

- Test each of the models on the appropriate dataset and store values


In [31]:
# set up test data
X_test = test_urinary_data.drop(columns=['diagnosis'])
y_test = test_urinary_data['diagnosis']

# Random Forest
rf_pred = rf.predict(X_test)
rf_acc = accuracy_score(y_test, rf_pred)
print("RF test accuracy: ", rf_acc)

# XGBoost
xgb_pred = xgb.predict(X_test)
xgb_acc = accuracy_score(y_test, xgb_pred)
print("XGB test accuracy: ", xgb_acc)

# SVM
svm_pred = svm.predict(X_test)
svm_acc = accuracy_score(y_test, svm_pred)
print("SVM test accuracy: ", svm_acc)

# Gaussian Naive Bayes
gnb_pred = gnb.predict(X_test)
gnb_acc = accuracy_score(y_test, gnb_pred)
print("GNB test accuracy: ", gnb_acc)

# KNearestNeighbors
knn_pred = knn.predict(X_test)
knn_acc = accuracy_score(y_test, knn_pred)
print("KNN test accuracy: ", knn_acc)

RF test accuracy:  0.577639751552795
XGB test accuracy:  0.13664596273291926
SVM test accuracy:  0.5714285714285714
GNB test accuracy:  0.515527950310559
KNN test accuracy:  0.5652173913043478


# Significance Analysis of Results

- Do the McNemar test here for each of the models (10?) to compare

# Comparison Across Datasets

- take urinary dataset models and try to predict on other dataset's test samples and vice versa
    - draw conclusions on the accuracy percentage and based on this, we can determine if the urinary biomarkers
        - are generalizeable to use on other data for predicting pancreatic cancer

# ------------- NON URINARY DATA WORK --------

### Normal pancreas RNA-seq data

In [33]:
# Normal pancreas dataset 1
markers = ['LYVE1','TFF1','REG1B','GPX1']

rawPancNormData1 = pd.read_csv("pancreaticNormalSeqData/GSE205163_znf808_ko_raw_counts_S0-S4.tsv",  sep='\t')
filtRawPancNormData1 = rawPancNormData1[rawPancNormData1['Gene'].str.endswith(('LYVE1','REG1B','TFF1','GPX1'))]
filtRawPancNormData1 = filtRawPancNormData1.set_index('Gene')
filtRawPancNormData1 = filtRawPancNormData1.transpose()
filtRawPancNormData1 = filtRawPancNormData1.reset_index(drop=True)
filtRawPancNormData1.columns = markers
print(filtRawPancNormData1)

    LYVE1  TFF1  REG1B  GPX1
0       1     0      0  5925
1       0     0      0  6520
2       0    14      0  4899
3       0     7      1  5882
4       0     4      0  4279
5       0    12      0  3314
6       0     1      0  4659
7       0     0      0  4370
8       0    11      0  4033
9       0     5      0  2866
10      1     0      0  7057
11      1     0      0  5737
12      0     6      0  5096
13      0     8      2  4481
14      2     4      0  3867
15      0    11      0  4731
16      0     4      0  4539
17      0     2      0  4218
18      1     6      0  3802
19      0    10      0  3586
20      3     0      0  8534
21      0     0      0  7313
22      0    12      0  7994
23      0     8      1  5321
24      0     3      0  4551
25      3     5      0  3980
26      0     0      0  3512
27      0     3      0  3196
28      0     7      0  2899
29      2     9      0  2725


In [34]:
# Normal pancreas dataset 2
rawPancNormData2 = pd.read_csv("pancreaticNormalSeqData/GSE216854_normalized_counts.txt", sep='\t')
filtRawPancNormData2 = rawPancNormData2[rawPancNormData2['gene'].isin(markers)]
filtRawPancNormData2 = filtRawPancNormData2.set_index('gene')
filtRawPancNormData2 = filtRawPancNormData2.transpose()
filtRawPancNormData2 = filtRawPancNormData2.reset_index(drop=True)
print(filtRawPancNormData2)

gene     REG1B         GPX1       TFF1     LYVE1
0     0.000000  1338.152011   0.743831  1.487662
1     0.000000  1317.741727  11.420428  0.000000
2     0.000000  1233.423715   7.913809  0.000000
3     0.000000   987.178392   7.446389  2.127540
4     0.000000   914.030573  10.528101  0.000000
5     0.000000   791.104182  18.776207  1.251747
6     0.000000   911.572944   5.683123  2.273249
7     0.000000   833.529266   6.872444  0.000000
8     0.000000   781.476317   1.154322  1.154322
9     0.000000  1594.628915   0.000000  0.000000
10    0.000000  1626.013137   0.832999  0.000000
11    0.000000  1660.088216   0.996452  0.000000
12    0.000000  2062.778781   0.000000  0.000000
13    0.000000  1804.216658   0.000000  0.000000
14    0.000000  1785.490649   0.000000  0.000000
15    1.186684  3163.700430   2.373369  2.373369
16    0.000000  2551.999220   0.000000  0.000000
17    0.000000  2735.777773   0.857342  0.000000
18    0.000000   967.223814   1.952673  0.000000
19    0.000000  1044

In [35]:
# Normal pancreas dataset 3
rawPancNormData3 = pd.read_csv("pancreaticNormalSeqData/GSE228662_RNA_raw_read_counts.tsv", sep='\t')
filtRawPancNormData3 = rawPancNormData3[rawPancNormData3['symbol'].isin(markers)]
filtRawPancNormData3 = filtRawPancNormData3.set_index('symbol')
filtRawPancNormData3 = filtRawPancNormData3.drop(columns=['chrom','start','end','gene'])
filtRawPancNormData3 = filtRawPancNormData3.transpose()
filtRawPancNormData3 = filtRawPancNormData3.reset_index(drop=True)
print(filtRawPancNormData3)

symbol  LYVE1  TFF1  REG1B  GPX1
0           0   117      0  2187
1           0   189      0  2722
2           1   127      0  1834
3           1    87      0  1795
4           0   122      0  1796
..        ...   ...    ...   ...
62          0    40      0  1087
63          2    25      0  1412
64          0    14      0  1539
65          1    26      0  1208
66          0    22      0  1436

[67 rows x 4 columns]


In [36]:
# Merging all 3 normal pancreas datasets
allPancNormData = pd.concat([filtRawPancNormData1, filtRawPancNormData2, filtRawPancNormData3], sort=False)
print(allPancNormData)

    LYVE1  TFF1  REG1B    GPX1
0     1.0   0.0    0.0  5925.0
1     0.0   0.0    0.0  6520.0
2     0.0  14.0    0.0  4899.0
3     0.0   7.0    1.0  5882.0
4     0.0   4.0    0.0  4279.0
..    ...   ...    ...     ...
62    0.0  40.0    0.0  1087.0
63    2.0  25.0    0.0  1412.0
64    0.0  14.0    0.0  1539.0
65    1.0  26.0    0.0  1208.0
66    0.0  22.0    0.0  1436.0

[139 rows x 4 columns]


In [59]:
# Processing normal pancreas data

# find negatives, NaN/null
# impute data if NaN/null

### Pancreatic cancer RNA-seq data

In [43]:
rawPancCancData1 = pd.read_csv("pancreaticCancerSeqData/GSE232860_allsamples.deseq.normalized.counts.csv")
#print(rawPancCancData1.shape)


#Slow loading, 48,553 rows
#rawPancCancData2 = pd.read_excel("pancreaticCancerSeqData/GSE245306_FKPM.xlsx")
#print(rawPancCancData2.shape)

#59050 rows
rawPancCancData3 = pd.read_csv("pancreaticCancerSeqData/tumor.counts.sub.tsv", sep='\t')
#print(rawPancCancData3)

In [65]:
#Only has Reg1, not Reg1B or Reg1A, so we take Reg1 and apply it to both rows
rowsToKeep = ["Gpx1", "Lyve1", "Reg1", "Tff1", "Reg1"]
rawPancCancData1.rename(columns={'Unnamed: 0': 'GeneNames'}, inplace=True)
print(rawPancCancData1.shape)

filtRawPancCancData1 = rawPancCancData1[rawPancCancData1['GeneNames'].isin(rowsToKeep)]
filtRawPancCancData1 = filtRawPancCancData1.transpose()
filtRawPancCancData1.reset_index(inplace=True, drop=True)
filtRawPancCancData1.columns = filtRawPancCancData1.iloc[0]
filtRawPancCancData1 = filtRawPancCancData1[1:]

filtRawPancCancData1["Reg1B"] = filtRawPancCancData1["Reg1"].copy()
filtRawPancCancData1.rename(columns={'Reg1': 'Reg1A'}, inplace=True)

# Impute missing values in each column if needed
filtRawPancCancData1 = filtRawPancCancData1.fillna(filtRawPancCancData1.median())
print(filtRawPancCancData1)





(17966, 23)
0        Tff1          Reg1A       Lyve1          Gpx1          Reg1B
1   86.519650    1152.208802  111.477242   7915.716067    1152.208802
2   99.259037    2032.652444   81.133473   6992.151782    2032.652444
3   95.638935    6535.034757   80.722771   8339.013202    6535.034757
4   42.385798   10158.844770   61.860354   7007.403411   10158.844770
5    6.260937   39103.310850  152.766873  10529.644580   39103.310850
6   33.793363    6810.356633  189.839188   7140.338886    6810.356633
7    4.244787    3628.231888  217.545346   8391.944362    3628.231888
8    2.204342   60100.287620  598.478904   6179.873328   60100.287620
9    4.750487    3313.939555  153.915771   7932.362771    3313.939555
10  21.847610   16311.226750  294.942730   6057.746312   16311.226750
11  42.165889    2393.872495  151.413873   8880.711134    2393.872495
12  35.099226    1474.167487  128.697162   6995.958204    1474.167487
13   0.000000   82462.293950   26.049017   8902.410494   82462.293950
14   7.9